# Education Desert maps

10, 20, 50 mi radii around higher education institutions. *What parts of the country are left out?*

In [ ]:
import pandas as pd
import geopandas as gpd
import geoplot as gplt
import geoplot.crs as gcrs
import matplotlib.pyplot as plt
from pathlib import Path

from tools import tree
from datetime import datetime as dt
today = dt.today().strftime("%d-%b-%y")

today

In [ ]:
RAW_DATA = Path("../data/raw/")
INTERIM_DATA = Path("../data/interim/")
PROCESSED_DATA = Path("../data/processed/")
FINAL_DATA = Path("../data/final/")
EXTERNAL_DATA = Path("../data/external/")

In [ ]:
tree(PROCESSED_DATA)

In [ ]:
county_data = pd.read_csv(PROCESSED_DATA / 'counties.csv')

In [ ]:
county_data.head().T

In [ ]:
institutions_data = pd.read_csv(PROCESSED_DATA / 'institutional_characteristics.csv')

In [ ]:
institutions_data.head().T

In [ ]:
county_shapes = gpd.read_file(PROCESSED_DATA / 'geodata' / 'tl_2019_us_county.shp')

In [ ]:
county_shapes.head().T

In [ ]:
import us

In [ ]:
contiguous_fips = [state.fips for state in us.STATES_CONTIGUOUS]
mask_contiguous_fips = county_shapes['STATEFP'].isin(contiguous_fips)
county_shapes = county_shapes[mask_contiguous_fips]

In [ ]:
contiguous_states = [state.name for state in us.STATES_CONTIGUOUS]
mask_contiguous_states = institutions_data['fips_state_code'].isin(contiguous_states)
institutions_data = institutions_data[mask_contiguous_states]

In [ ]:
geo_institutions = gpd.GeoDataFrame(institutions_data, geometry=gpd.points_from_xy(institutions_data['longitude'], institutions_data['latitude']))

In [ ]:
geo_institutions.crs

In [ ]:
county_shapes.crs

In [ ]:
geo_institutions.crs = county_shapes.crs

In [ ]:
geo_institutions.plot();

[EPSG: 4269](https://epsg.io/4269) uses degrees as its units of measure, we need to change that to meters so we can create _buffers_ of 10, 25, 50 miles around each institution.

[EPSG: 3857](https://epsg.io/3857)

In [ ]:
geo_institutions_in_meters = geo_institutions.to_crs(epsg=3857)

In [ ]:
geo_institutions_in_meters.head()

In [ ]:
miles = 1609 # meters

In [ ]:
institutions_10mi_radius = geo_institutions_in_meters.copy()
institutions_10mi_radius['geometry'] = institutions_10mi_radius['geometry'].buffer(10 * miles)

In [ ]:
institutions_25mi_radius = geo_institutions_in_meters.copy()
institutions_25mi_radius['geometry'] = institutions_10mi_radius['geometry'].buffer(25 * miles)

In [ ]:
institutions_50mi_radius = geo_institutions_in_meters.copy()
institutions_50mi_radius['geometry'] = institutions_50mi_radius['geometry'].buffer(50 * miles)

In [ ]:
institutions_10mi_radius.to_crs(epsg=4269, inplace=True)
institutions_25mi_radius.to_crs(epsg=4269, inplace=True)
institutions_50mi_radius.to_crs(epsg=4269, inplace=True)

In [ ]:
institutions_10mi_radius.plot();

In [ ]:
institutions_25mi_radius.plot();

In [ ]:
institutions_50mi_radius.plot();

https://geopandas.org/set_operations.html

In [ ]:
gpd.overlay(county_shapes, institutions_10mi_radius, how = 'difference').plot();

In [ ]:
gpd.overlay(county_shapes, institutions_25mi_radius, how = 'difference').plot();

In [ ]:
gpd.overlay(county_shapes, institutions_50mi_radius, how = 'difference').plot();

but overlay actually returns a geodataframe which we can use with geoplot!

In [ ]:
gpd.overlay(county_shapes, institutions_50mi_radius, how = 'difference')

TODO

attach county data, share underrepresented, hh income, median age, to these dataframes.

# Data preparation

In [ ]:
county_data.head()

In [ ]:
county_data['share_underrepresented'] = (county_data['black_alone'] + 
    county_data['american_indian_and_alaska_native'] + 
    county_data['native_hawaiian_and_pacific_islander'] + 
    county_data['latino_alone']) / county_data['universe']

In [ ]:
county_data['geoid'] = county_data['geoid'].astype(str).str.zfill(5)

county_data = county_data[['geoid', 'name', 'universe', 'share_underrepresented']]
county_data.set_index('geoid', inplace = True)

In [ ]:
median_age = pd.read_csv(EXTERNAL_DATA / 'processed' / 'acs5_2018_medianage_counties.csv')
median_hh_income = pd.read_csv(EXTERNAL_DATA / 'processed' / 'acs5_2018_medianhouseholdincome_counties.csv')

In [ ]:
median_age.head()

In [ ]:
median_hh_income.head()

In [ ]:
# fix geoid
median_age['geoid'] = median_age['geoid'].astype(str).str.zfill(5)
median_hh_income['geoid'] = median_hh_income['geoid'].astype(str).str.zfill(5)

In [ ]:
median_age = median_age[['geoid', 'median', 'male', 'female']]
median_age.columns = ['geoid', 'median_age', 'median_age_male', 'median_age_female']
median_age.set_index('geoid', inplace = True)

In [ ]:
median_hh_income = median_hh_income[['geoid', 'median']]
median_hh_income.columns = ['geoid', 'median_hh_income']
median_hh_income.set_index('geoid', inplace=True)

In [ ]:
county_shapes = county_shapes[['GEOID', 'geometry']]
county_shapes.set_index("GEOID", inplace = True)

In [ ]:
working_gdf = county_shapes.join(county_data).join(median_age).join(median_hh_income)

working_gdf.head()

In [ ]:
ten_miles = gpd.overlay(working_gdf, institutions_10mi_radius, how = 'difference')
twentyfive_miles = gpd.overlay(working_gdf, institutions_25mi_radius, how = 'difference')
fifty_miles = gpd.overlay(working_gdf, institutions_50mi_radius, how = 'difference')

In [ ]:
ten_miles_inter = gpd.overlay(working_gdf, institutions_10mi_radius, how = 'intersection')
twentyfive_miles_inter = gpd.overlay(working_gdf, institutions_25mi_radius, how = 'intersection')
fifty_miles_inter = gpd.overlay(working_gdf, institutions_50mi_radius, how = 'intersection')

In [ ]:
gplt.choropleth(ten_miles, projection=gcrs.WebMercator(), hue = 'share_underrepresented', figsize=(12,12),)

In [ ]:
import matplotlib.colors as mc

In [ ]:
univ_norm = mc.Normalize(ten_miles['universe'].min(), ten_miles['universe'].quantile(0.90))

In [ ]:
proj = gcrs.AlbersEqualArea()

ax = gplt.polyplot(
    county_shapes,     
    zorder=-1,
    projection = proj,
    linewidth=1,
    edgecolor='white',
    facecolor='lightgray',
    figsize=(12, 12),
)
gplt.choropleth(ten_miles, hue = 'share_underrepresented', ax = ax)

In [ ]:
proj = gcrs.AlbersEqualArea()

ax = gplt.polyplot(
    county_shapes,     
    zorder=-1,
    projection = proj,
    linewidth=1,
    edgecolor='white',
    facecolor='lightgray',
    figsize=(12, 12),
)
gplt.choropleth(twentyfive_miles, hue = 'share_underrepresented', ax = ax)

In [ ]:
proj = gcrs.AlbersEqualArea()

ax = gplt.polyplot(
    county_shapes,     
    zorder=-1,
    projection = proj,
    linewidth=1,
    edgecolor='white',
    facecolor='lightgray',
    figsize=(12, 12),
)
gplt.choropleth(fifty_miles, hue = 'share_underrepresented', ax = ax)

***
EXTRA (this wasn't covered in the youtube tutorial)

In [ ]:
def make_cool_map(gdf, var_of_interest, ax, projection = gcrs.AlbersEqualArea(),):
    """makes a cool choropleth map from our data
    """
    proj = gcrs.AlbersEqualArea()

    ax = gplt.polyplot(
        county_shapes,     
        zorder=-1,
        projection = proj,
        linewidth=1,
        edgecolor='white',
        facecolor='lightgray',
        ax = ax,
    )
    
    gplt.choropleth(
        gdf, 
        hue = var_of_interest, 
        ax = ax, 
    )

In [ ]:
proj = gcrs.WebMercator()
f, axarr = plt.subplots(2, 2, figsize=(12, 12), subplot_kw={'projection': proj})

plt.suptitle('Some cool maps', fontsize=16)
plt.subplots_adjust(top=0.95)

make_cool_map(fifty_miles, 'share_underrepresented', axarr[0][0])
axarr[0][0].set_title('Share URM')

make_cool_map(fifty_miles_inter, 'share_underrepresented', axarr[0][1])
axarr[0][1].set_title('Share URM')

make_cool_map(fifty_miles, 'median_hh_income', axarr[1][0])
axarr[1][0].set_title('Median household income')

make_cool_map(fifty_miles_inter, 'median_hh_income', axarr[1][1])
axarr[1][1].set_title('Median household income')

plt.savefig("test.png", bbox_inches='tight')